# Checkpointing and early stopping

This section covers two techniques for making training more economical: checkpointing and early stopping. Checkpointing is useful when a model overtrains and diverges, and we want to recover the model weights at the point of convergence without the additional cost of retraining. You can think of early stopping as an extension to checkpointing. We have a monitoring system detect divergence at the earliest moment that it occurs, and then we stop training, saving additional cost when recovering a checkpoint at the point of divergence.


## Check Pointing
Checkpointing is periodically saving the learned model parameters and current hyperparameter values during training. There are two reasons for doing this:
1. To be able to resume training of a model where you left off, instead of restarting the training from the beginning
2. To identify a past point in training where the model gave the best results

In the first case, we might want to split training across sessions as a way of managing resources. For example, we might reserve (or be authorized) one hour a day for training. At the end of the one-hour training each day, the training is checkpointed. The following day, training is resumed by restoring from the checkpoint. For example, you might be working in a research organization that has a fixed budget for computing expenses, and your team is experimenting with training a model with substantial computing costs. To manage the budget, your team might be allocated a limit of daily computing expenses.

Why wouldn’t saving the model’s weights and biases be enough? In neural networks, some hyperparameter values will dynamically change, such as the learning rate and decay. We would want to resume at the same hyperparameter values at the time the training was paused.

In another scenario, we might implement continuous learning as a part of a continuous integration and delivery (CI/CD) process. In this scenario, new labeled
images are **continuously added** to the training data, and we want to only **incrementally retrain the model instead of retraining from scratch** on each integration cycle.

 In the second case, we might want to find the best result after a model has trained past the best optimum, and started to diverge and/or overfit. We would not want to start retraining from scratch with fewer epochs (or other hyperparameter changes), but instead identify the epoch that achieved the best results, and restore (set) the learned model parameters to those that were checkpointed at the end of that epoch.




In [1]:
from keras.callbacks import ModelCheckpoint
file_path = "./saved_model/mymodel_{epoch:02d}.ckpt"
checkpoint = ModelCheckpoint(file_path)

In [15]:
from keras import Sequential
from keras.layers import Dense
import tensorflow as tf
model = Sequential([
    Dense(3, activation="relu", input_shape=(28,3))
])
model.compile(loss="mse", optimizer="adam", metrics=tf.keras.metrics.RootMeanSquaredError())
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 28, 3)             12        
                                                                 
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [16]:
import numpy as np
X = np.random.randn(28,28,3)
X = tf.convert_to_tensor(X)
y = 2*X**2 + 0.5*X +10
model.fit(X, y, epochs=10, callbacks=[checkpoint])

Epoch 1/10
1/1 [==============================] - 1s 587ms/step - loss: 144.4981 - root_mean_squared_error: 12.0207
Epoch 2/10
1/1 [==============================] - 0s 230ms/step - loss: 144.4720 - root_mean_squared_error: 12.0197
Epoch 3/10
1/1 [==============================] - 0s 216ms/step - loss: 144.4459 - root_mean_squared_error: 12.0186
Epoch 4/10
1/1 [==============================] - 0s 237ms/step - loss: 144.4198 - root_mean_squared_error: 12.0175
Epoch 5/10
1/1 [==============================] - 0s 244ms/step - loss: 144.3936 - root_mean_squared_error: 12.0164
Epoch 6/10
1/1 [==============================] - 0s 238ms/step - loss: 144.3674 - root_mean_squared_error: 12.0153
Epoch 7/10
1/1 [==============================] - 0s 239ms/step - loss: 144.3412 - root_mean_squared_error: 12.0142
Epoch 8/10
1/1 [==============================] - 0s 224ms/step - loss: 144.3150 - root_mean_squared_error: 12.0131
Epoch 9/10
1/1 [==============================] - 0s 226ms/step - loss: 

Alternatively, we can save the current best checkpoint with the parameter **save_best_only=True** and the **parameter monitor to the measurement** to base the decision on.

For example, if the parameter monitor is set to val_acc, it will write a checkpoint only if the validation accuracy is higher than the last saved checkpoint. If the parameter is set to val_loss, it will write a checkpoint only if the validation loss is lower than the last saved checkpoint:


In [22]:
checkpoint_val_acc = ModelCheckpoint(filepath=file_path, save_best_only=True, monitor="val_acc")
model2 = Sequential([
    Dense(3, activation='relu', input_shape=(28,3))])
model2.compile(loss='mse', optimizer="adam", metrics=[tf.keras.metrics.RootMeanSquaredError()])
model2.fit(X, y, epochs=10, callbacks=[checkpoint_val_acc])

Epoch 1/10
1/1 [==============================] - 0s 287ms/step - loss: 144.3066 - root_mean_squared_error: 12.0128
Epoch 2/10
1/1 [==============================] - 0s 3ms/step - loss: 144.2818 - root_mean_squared_error: 12.0117
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 144.2569 - root_mean_squared_error: 12.0107
Epoch 4/10
1/1 [==============================] - 0s 2ms/step - loss: 144.2321 - root_mean_squared_error: 12.0097
Epoch 5/10
1/1 [==============================] - 0s 2ms/step - loss: 144.2072 - root_mean_squared_error: 12.0086
Epoch 6/10
1/1 [==============================] - 0s 2ms/step - loss: 144.1824 - root_mean_squared_error: 12.0076
Epoch 7/10
1/1 [==============================] - 0s 2ms/step - loss: 144.1575 - root_mean_squared_error: 12.0066
Epoch 8/10
1/1 [==============================] - 0s 2ms/step - loss: 144.1326 - root_mean_squared_error: 12.0055
Epoch 9/10
1/1 [==============================] - 0s 991us/step - loss: 144.1077 - roo

In [21]:
X.shape

TensorShape([28, 28, 3])

## Early stopping
An early stop is setting a condition upon which training is *terminated earlier than the set limits* (for example, number of epochs).

This is generally set to conserve resources and/or prevent overtraining when a goal objective is reached, such as a level of accuracy or convergence on evaluation loss.

For example, we might set a training for 20 epochs, which average 30 minutes each, for a total of 10 hours. But if the objective is met after 8 epochs, it would be ideal to terminate the training, saving 6 hours of resources.

An early stop is specified in a manner similar to a checkpoint. An EarlyStopping object is instantiated and configured with a target goal, and passed to the callbacks parameter of the fit() method. In this example, training will be stopped early only if the validation loss stops, reducing from the previous epoch:








In [27]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss')
model3 = Sequential([Dense(3, activation='relu', input_shape=(28,3))])
model3.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model3.fit(X, y, epochs=100, callbacks=[early_stop, checkpoint_val_acc])

Epoch 1/100
1/1 [==============================] - 0s 175ms/step - loss: 144.7578 - root_mean_squared_error: 12.0315
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 144.7335 - root_mean_squared_error: 12.0305


In [28]:
early_stop = EarlyStopping(monitor='loss',  patience=3)
model4 = Sequential([Dense(3, activation='relu', input_shape=(28,3))])
model4.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model4.fit(X, y, epochs=100, callbacks=[early_stop, checkpoint_val_acc])

Epoch 1/100
1/1 [==============================] - 0s 167ms/step - loss: 141.0353 - root_mean_squared_error: 11.8758
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 141.0090 - root_mean_squared_error: 11.8747
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 140.9826 - root_mean_squared_error: 11.8736
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 140.9563 - root_mean_squared_error: 11.8725
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 140.9300 - root_mean_squared_error: 11.8714
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 140.9037 - root_mean_squared_error: 11.8703
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 140.8773 - root_mean_squared_error: 11.8692
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: 140.8510 - root_mean_squared_error: 11.8681
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 140.824

<img src="img_10.png" />

<img src="img_11.png" />

The parameter:
 1. *patience* specifies a minimum number of epochs without improvement before early stopping, and
 2. *min_delta* specifies a minimum threshold to determine whether the model improved.